In [1]:
%matplotlib inline
#
from lrgsglib import *
#
move_to_rootf(pathname='lrg_eegfc')
#
from lrg_eegfc import *
from ipywidgets import interact, FloatSlider

Current working directory: /home/opisthofulax/Documents/research+/brain_network/lrg_eegfc


### data reader

In [2]:
data_dict, int_label_pos_map = load_data_dict(
    mat_path, patients_list, phase_labels, param_keys_list
)

### interactive multiscale clustering

In [3]:
patient = 'Pat_02'
phase = 'rsPre'

In [4]:
correlation_protocol = dict(threshold=0,spectral_cleaning=False)
#
data_pat_phase = data_dict[patient][phase]
data_pat_phase_ts = data_pat_phase['data']
fs = data_pat_phase['fs']
filter_order = data_pat_phase['filter_order']
pin_labels = int_label_pos_map[patient]['label']

In [5]:
# plot settings
kw_thline = dict(color='b', linestyle='--', label=r'$\mathcal{D}_{\rm th}$')
kw_draw_graph = dict(node_size=100, font_size=8, with_labels=True)
kw_dentro = dict(above_threshold_color='k', leaf_font_size=5, orientation='top')

In [6]:
custom_th = None
jump_num = 0
# Build correlation matrix and find threshold jumps
corr_mat = build_corr_network(data_pat_phase_ts, **correlation_protocol)
Th, jumps = find_threshold_jumps(nx.from_numpy_array(corr_mat))
# Select threshold just below the selected jump and get the giant component
cm_tmp = corr_mat.copy()
cm_tmp[cm_tmp < Th[jumps[jump_num]]] = 0
G, rnodes = get_giant_component_leftoff(nx.from_numpy_array(cm_tmp))
label_dict = {k: v for k, v in pin_labels.to_dict().items() if k not in rnodes}
# Compute the linkage matrix and optimal threshold
spect, L, rho, Trho, tau = compute_laplacian_properties(G, tau=None)
dists = squareform(Trho)
lnkgM, label_list, _ = compute_normalized_linkage(dists, G, method='ward')
clTh, *_ = compute_optimal_threshold(lnkgM, scaling_factor=0.98)
clTh = custom_th or clTh
# Prepare for plotting
node_list = list(G.nodes())
labels_for_dendro = [label_dict[n] for n in node_list]
widths = [G[u][v]['weight'] for u, v in G.edges()]
edge_colors = ['r' if d['weight'] < 0 else 'b' for _,_,d in G.edges(data=True)]

In [ ]:
# Plotting function
def plot_dendro_and_graph(clTh):
    fig, ax = plt.subplots(1, 2, figsize=(15, 7.5))
    
    # Dendrogram first (to get the leaf colors)
    dendro = dendrogram(
        lnkgM,
        ax=ax[0],
        color_threshold=clTh,
        labels=labels_for_dendro,
        **kw_dentro
    )
    tmin = lnkgM[:, 2][0] * 0.8
    tmax = lnkgM[:, 2][-1] * 1.01
    ax[0].set_yscale('log')
    ax[0].axhline(clTh, **kw_thline)
    ax[0].set_ylim(tmin, tmax)
    # ax[0].set_ylabel(r'$\\mathcal{D}/\\mathcal{D}_{max}$')
    
    # Get the leaf colors from dendrogram and map to nodes
    leaf_label_colors = {lbl: col for lbl, col in zip(dendro['ivl'], dendro['leaves_color_list'])}
    
    # Map node colors based on dendrogram leaf colors
    node_colors = [leaf_label_colors.get(label_dict[n], 'gray') for n in node_list]
    
    pos = nx.kamada_kawai_layout(G)
    
    # Network plot
    # pos = nx.kamada_kawai_layout(G)
    nx.draw(G, ax=ax[1], width=widths, edge_color=edge_colors,
            labels=label_dict,
            node_color=node_colors, pos=pos, **kw_draw_graph)
    plt.show()
# Logarithmic slider for clTh
def log_slider_to_clTh(log_clTh):
    return 10**log_clTh
# 
min_th = lnkgM[:, 2][0]
max_th = lnkgM[:, 2][-1]
min_exp = np.log10(min_th)
max_exp = np.log10(max_th)
#
log_slider = FloatSlider(
    value=np.log10(clTh),
    min=min_exp,
    max=max_exp,
    step=(max_exp - min_exp) / 100,
    description='log10(clTh)',
    readout_format='.2f'
    )
#

In [8]:
interact(lambda log_clTh: plot_dendro_and_graph(log_slider_to_clTh(log_clTh)), log_clTh=log_slider)

interactive(children=(FloatSlider(value=-6.240624493197499, description='log10(clTh)', max=-0.0043213737826426…

<function __main__.<lambda>(log_clTh)>